In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys


In [ ]:
!pip install mlflow

In [5]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))
from model_building import *

In [6]:
df_fraud = pd.read_csv('/content/fraud_data_final.csv')

In [7]:
df_fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,age,ip_address,class,ip_address_int,transaction_velocity_hours,hour_of_day,day_of_week,source_Direct,source_SEO,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,-0.160204,QVPSPJUOCKZAR,0.679914,43.173.1.96,0,-1.136880,-0.136057,-1.377455,0.991020,False,True,False,False,False,False,True
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,-1.142592,EOGFQPIZPYXFZ,2.304476,20.225.83.219,0,-1.443207,-1.571877,-1.522122,-1.501259,False,False,False,False,False,False,False
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,-1.197169,YSSKYOSJHPPLJ,2.304476,156.64.132.28,1,0.375916,-1.577617,0.937208,-0.005891,False,True,False,False,True,False,True
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,0.385567,ATGTXKYKUDUQN,0.911994,228.234.6.235,0,1.352348,-1.420213,0.213876,-1.501259,False,True,False,False,False,True,True
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,0.112681,NAUITBZFJKHWW,1.376155,24.197.75.141,0,-1.390927,-0.182509,0.937208,-0.504347,False,False,False,False,False,True,True


In [8]:
df_merged = pd.read_csv('/content/fraud_ip_to_country_merged_.csv')

In [9]:
df_merged.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,ip_address_int,lower_bound_ip_address,upper_bound_ip_address,country,lower_bound_ip_address_int,upper_bound_ip_address_int
0,247547,2015-06-28 03:00:34,2015-08-09 03:57:29,47,KIXYSVCHIPQBR,SEO,Safari,F,30,1.0.6.112,0,16778864,1.0.4.0,1.0.7.255,Australia,16778240.0,16779263.0
1,220737,2015-01-28 14:21:11,2015-02-11 20:28:28,15,PKYOWQKWGJNJI,SEO,Chrome,F,34,1.0.253.61,0,16842045,1.0.128.0,1.0.255.255,Thailand,16809984.0,16842751.0
2,390400,2015-03-19 20:49:09,2015-04-11 23:41:23,44,LVCSXLISZHVUO,Ads,IE,M,29,1.1.3.136,0,16843656,1.1.2.0,1.1.3.255,China,16843264.0,16843775.0
3,69592,2015-02-24 06:11:57,2015-05-23 16:40:14,55,UHAUHNXXUADJE,Direct,Chrome,F,30,1.2.118.236,0,16938732,1.2.64.0,1.2.127.255,China,16924672.0,16941055.0
4,174987,2015-07-07 12:58:11,2015-11-03 04:04:30,51,XPGPMOHIDRMGE,SEO,Chrome,F,37,1.2.248.208,0,16971984,1.2.128.0,1.2.255.255,Thailand,16941056.0,16973823.0


In [10]:
df_for_model = df_fraud.merge(df_merged[['user_id', 'country']], on='user_id', how='left')
df_for_model.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,age,ip_address,class,ip_address_int,transaction_velocity_hours,hour_of_day,day_of_week,source_Direct,source_SEO,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M,country
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,-0.160204,QVPSPJUOCKZAR,0.679914,43.173.1.96,0,-1.136880,-0.136057,-1.377455,0.991020,False,True,False,False,False,False,True,Japan
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,-1.142592,EOGFQPIZPYXFZ,2.304476,20.225.83.219,0,-1.443207,-1.571877,-1.522122,-1.501259,False,False,False,False,False,False,False,United States
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,-1.197169,YSSKYOSJHPPLJ,2.304476,156.64.132.28,1,0.375916,-1.577617,0.937208,-0.005891,False,True,False,False,True,False,True,United States
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,0.385567,ATGTXKYKUDUQN,0.911994,228.234.6.235,0,1.352348,-1.420213,0.213876,-1.501259,False,True,False,False,False,True,True,NaN
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,0.112681,NAUITBZFJKHWW,1.376155,24.197.75.141,0,-1.390927,-0.182509,0.937208,-0.504347,False,False,False,False,False,True,True,United States


In [11]:
df_for_model.drop(['user_id', 'signup_time', 'purchase_time', 'device_id', 'ip_address'], axis=1, inplace=True)

In [12]:
df_for_model.head()

,purchase_value,age,class,ip_address_int,transaction_velocity_hours,hour_of_day,day_of_week,source_Direct,source_SEO,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M,country
0,-0.160204,0.679914,0,-1.136880,-0.136057,-1.377455,0.991020,False,True,False,False,False,False,True,Japan
1,-1.142592,2.304476,0,-1.443207,-1.571877,-1.522122,-1.501259,False,False,False,False,False,False,False,United States
2,-1.197169,2.304476,1,0.375916,-1.577617,0.937208,-0.005891,False,True,False,False,True,False,True,United States
3,0.385567,0.911994,0,1.352348,-1.420213,0.213876,-1.501259,False,True,False,False,False,True,True,NaN
4,0.112681,1.376155,0,-1.390927,-0.182509,0.937208,-0.504347,False,False,False,False,False,True,True,United States


## Filling Nan Values of country column with unknown

In [13]:
df_for_model['country'].fillna('Unknown', inplace=True)

<ipython-input-13-d50e88ef1c99>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_for_model['country'].fillna('Unknown', inplace=True)


In [14]:
# Calculate the frequency of each country
country_freq = df_for_model['country'].value_counts()

# Map the frequencies to the country column
df_for_model['country_freq'] = df_for_model['country'].map(country_freq)

df_for_model.head()

,purchase_value,age,class,ip_address_int,transaction_velocity_hours,hour_of_day,day_of_week,source_Direct,source_SEO,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M,country,country_freq
0,-0.160204,0.679914,0,-1.136880,-0.136057,-1.377455,0.991020,False,True,False,False,False,False,True,Japan,7306
1,-1.142592,2.304476,0,-1.443207,-1.571877,-1.522122,-1.501259,False,False,False,False,False,False,False,United States,58049
2,-1.197169,2.304476,1,0.375916,-1.577617,0.937208,-0.005891,False,True,False,False,True,False,True,United States,58049
3,0.385567,0.911994,0,1.352348,-1.420213,0.213876,-1.501259,False,True,False,False,False,True,True,Unknown,21966
4,0.112681,1.376155,0,-1.390927,-0.182509,0.937208,-0.504347,False,False,False,False,False,True,True,United States,58049


In [17]:
df_for_model.drop('country', axis=1, inplace=True)

In [19]:
trainer = ModelTrainer(df_for_model, target_column='class')

In [20]:
results = {
     "Logistic Regression": trainer.train_logistic_regression(),
     "Decision Tree": trainer.train_decision_tree(),
     "Random Forest": trainer.train_random_forest(),
     "Gradient Boosting": trainer.train_gradient_boosting(),
     "Multi-Layer Perceptron": trainer.train_mlp()
 }

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.9057
Decision Tree Accuracy: 0.9051
Random Forest Accuracy: 0.9564
Gradient Boosting Accuracy: 0.9564


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
3400/3400 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.8243 - loss: 159.9313 - val_accuracy: 0.9062 - val_loss: 0.4597
Epoch 2/10
3400/3400 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.8890 - loss: 0.5340 - val_accuracy: 0.9062 - val_loss: 0.3104
Epoch 3/10
3400/3400 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9066 - loss: 0.3116 - val_accuracy: 0.9062 - val_loss: 0.3107
Epoch 4/10
3400/3400 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9058 - loss: 0.3121 - val_accuracy: 0.9062 - val_loss: 0.3105
Epoch 5/10
3400/3400 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9065 - loss: 0.3124 - val_accuracy: 0.9062 - val_loss: 0.3107
Epoch 6/10
3400/3400 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9068 - loss: 0.3163 - val_accuracy: 0.9062 - val_loss: 0.3104
Epoch 7/10
3400/3400 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9068 - loss: 0.3122 - val_accuracy: 0.9062 - val_loss: 0.3101
Epoch 8/10
3400/3400 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9053 - loss: 0.317